In [1]:
import pandas as pd
df = pd.read_csv('DataTrain/fixed_dataset.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'DataTrain/fixed_dataset.csv'

In [3]:
df.head()

,Unnamed: 0,text,label
0,0,"pilgub kemarin banyak boong nya , hasil nya di...",negative
1,1,"hti , wahabi , pks , fpi ikut2an menyerbu kiai...",negative
2,2,"malas makan oreo , suka bikin gigi hitam",negative
3,3,lagi asik menonton di bioskop ada iklan pemeri...,negative
4,4,adeh . di mana letak muka nya itu orang . tida...,negative


In [4]:
df.shape

(3444, 3)

In [5]:
df.label.value_counts()

negative    1148
positive    1148
neutral     1148
Name: label, dtype: int64

In [6]:
from Data_Cleansing import process_text

In [7]:
df['text_clean'] = df.text.apply(process_text)
df

,Unnamed: 0,text,label,text_clean
0,0,"pilgub kemarin banyak boong nya , hasil nya di...",negative,pilgub kemarin banyak boong nya hasil nya dima...
1,1,"hti , wahabi , pks , fpi ikut2an menyerbu kiai...",negative,hti wahabi pks fpi ikut2an menyerbu kiai said ...
2,2,"malas makan oreo , suka bikin gigi hitam",negative,malas makan oreo suka bikin gigi hitam
3,3,lagi asik menonton di bioskop ada iklan pemeri...,negative,lagi asik menonton di bioskop ada iklan pemeri...
4,4,adeh . di mana letak muka nya itu orang . tida...,negative,adeh di mana letak muka nya itu orang tidak bi...
...,...,...,...,...
3439,3439,pkb benarkan charly setia band daftar jadi caw...,neutral,pkb benarkan charly setia band daftar jadi caw...
3440,3440,pernah percaya kalau body lotion citra bikin b...,neutral,pernah percaya kalau body lotion citra bikin b...
3441,3441,"bu susi marah , menteri luhut perbolehkan kapa...",neutral,bu susi marah menteri luhut perbolehkan kapal ...
3442,3442,f - demokrat dorong upaya kemandirian energi n...,neutral,f demokrat dorong upaya kemandirian energi nas...


In [8]:
neg = df.loc[df['label'] == 'negative'].text_clean.tolist()
neu = df.loc[df['label'] == 'neutral'].text_clean.tolist()
pos = df.loc[df['label'] == 'positive'].text_clean.tolist()

In [9]:
neg_label = df.loc[df['label'] == 'negative'].label.tolist()
neu_label = df.loc[df['label'] == 'neutral'].label.tolist()
pos_label = df.loc[df['label'] == 'positive'].label.tolist()

In [10]:
total_data = pos + neu + neg
labels = pos_label + neu_label + neg_label

print("Positive : %s, Neutral: %s, Negative : %s" % (len(pos), len(neu), len(neg)))
print("Total data : %s" % len(total_data))

Positive : 1148, Neutral: 1148, Negative : 1148
Total data : 3444


In [11]:
import pickle
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict

max_features = 100000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(total_data)
with open ('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("tokenizer.pickle has succesfully created")

X = tokenizer.texts_to_sequences(total_data)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X)

X = pad_sequences(X)
with open('x_pad_sequences.pickle', 'wb') as handle:
    pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('x_pad_sequences.pickle has created')

tokenizer.pickle has succesfully created
x_pad_sequences.pickle has created


In [12]:
Y = pd.get_dummies(labels)
Y = Y.values
with open('y_labels.pickle', 'wb') as handle:
    pickle.dump(Y, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('y_labels.pickle has created')

y_labels.pickle has created


In [33]:
from sklearn.model_selection import train_test_split

file = open('x_pad_sequences.pickle', 'rb')
X = pickle.load(file)
file.close()

file = open('y_labels.pickle', 'rb')
Y = pickle.load(file)
file.close()

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)

# # Our vectorized labels
# y_train = y_train.astype('float32').reshape((-1,1))
# y_test = y_test.astype('float32').reshape((-1,1))
y_train = pd.read_csv('DataTrain/fixed_dataset.csv', index_col=[0])

In [31]:
import numpy as np
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras import optimizers
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Flatten
from keras import backend as K

In [34]:
embed_dim = 100

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(Flatten())
model.add(layers.Dense(3, activation='softmax'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

es = EarlyStopping(monitor='val_loss', mode='min', verbose = 1)
history = model.fit(X_train, y_train, epochs = 10, batch_size = 10, validation_data=(X_test, y_test), verbose = 1, callbacks=[es])

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 96, 100)           10000000  
                                                                 
 conv1d_10 (Conv1D)          (None, 92, 128)           64128     
                                                                 
 dense_34 (Dense)            (None, 92, 10)            1290      
                                                                 
 flatten_4 (Flatten)         (None, 920)               0         
                                                                 
 dense_35 (Dense)            (None, 3)                 2763      
                                                                 
 dense_36 (Dense)            (None, 1)                 4         
                                                                 
Total params: 10,068,185
Trainable params: 10,068,185

ValueError: Data cardinality is ambiguous:
  x sizes: 2755
  y sizes: 3444
Make sure all arrays contain the same number of samples.